In [ ]:
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from fractions import Fraction
import time
import sys

dyn_pendulum = {"g":9.81, "m": 1.0, "l": 1.0}
x0 = np.array([np.pi-0.1, 0.0])
v1 = np.array([1.0, 0.0])
v2 = np.array([0.0, 1.0])
x_dot = []
x_norm = []
# cum = np.array([0,0,0])

# def cosin 

def simple_pendulum (x0, dyn):
    g = dyn['g'] 
    l = dyn['l']
    #print (x0)
    return np.array([x0[1], (-g/l)*np.sin(x0[0])])


def RungeKutta (dyn, f, dt, x0):
    k1 = f(x0, dyn) #[x,y,z]*0.1 example
    k2 = f(x0+0.5*k1*dt,dyn)
    k3 = f(x0 + 0.5*k2*dt, dyn)
    k4 = f(x0 + k3*dt, dyn)
    
    x = x0 + ((k1 + 2*k2 + 2*k3 + k4)/6) *dt
    return x

def f_t (dyn, f, dt, x0, T):
    x = np.empty(shape=(len(x0),T))
    #print(x.shape)
    x[:, 0] = x0
    # print ('x:', x)
    # print ('x[:, 0]:', x[:, 0])
    # print ('x0:', x0)
    for i in range(1,T):
        x[:, i] = RungeKutta(dyn, f, dt, x[:, i-1])
    return x


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

x = f_t(dyn_pendulum, simple_pendulum, 0.001, x0, 500)

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.gca(projection="3d")
ax.plot3D(x[0], x[1])
plt.title('simple pendulum')
plt.draw()

In [ ]:
def sample_based_truncated_le (x, evolv, dt, scalmx, scalmn):
    IND = 0
    SUM = 0.0
    ITS = 0
    IND2 = 0

    DI = np.inf #any large number

    NPT = len(x.T)
    DIM = len(x)
    DT = dt
    SCALMX = scalmx
    SCALMN = scalmn
#     EVOLV = 5 #time from t0 to t1
#     EVOLV_physical = EVOLV/DT
    
    EVOLV_physical = evolv #time from t0 to t1
    EVOLV = int(EVOLV_physical/DT)
    #find nearest neighbor to the first data point
    Z = x.T
    
    for i in range(1, int(NPT)-(EVOLV)):
        D = 0.0
        #iterate through the array, compare first point with the rest
        D = np.linalg.norm(Z[IND] - Z[i])
        print (D)
        if ((D<DI) and (D>=SCALMN)):
            
            DI = D
            IND2 = i
            
    print (DI)
    SUM_list = []
    while ((IND + EVOLV < NPT) and (IND2 + EVOLV < NPT)): #40
        PT1 = Z[IND+EVOLV]
        PT2 = Z[IND2+EVOLV]
        DF = np.linalg.norm(PT1 - PT2)
        ITS = ITS + 1
        SUM_list.append(SUM)
        SUM = SUM + np.log2(DF/DI) / EVOLV_physical
        ZLYAP = SUM/ITS

        INDOLD = IND2
        ZMULT = 1.0
        ANGLMX = 0.3
        THMIN = 3.14
        while (True): #70

            for i in range (int(NPT)): #100
                III = int(np.absolute(i-(IND+EVOLV)))
                if (III >= 10):
                    #find angular change old to new vector
                    DNEW = 0.0
                    DNEW = np.linalg.norm(PT1 - Z[i])
                    if ((DNEW <= ZMULT*SCALMX) and (DNEW >= SCALMN)):
                        DOT = 0.0
                        DOT = np.dot((PT1 - Z[i]),(PT1 - PT2))
                        CTH = np.absolute(DOT/(DNEW*DF))
                        #print(CTH)
                        if (CTH > 1.0):
                            CTH = 1.0
                        TH = np.arccos(CTH)
                        #save smallest angular change so far
                        if TH <= THMIN:
                            THMIN = TH
                            DII = DNEW
                            IND2 = i
            if (THMIN < ANGLMX):
                #print ("here")
                break
            # look at longer distance if cant find replacement
            ZMULT = ZMULT + 1
            if (ZMULT > 5):
                #no replacement at 5*scale, double search angle, reset distance
                ZMULT = 1.0
                ANGLMX = 2.0*ANGLMX
            if (ANGLMX >= 3.14):
                IND2 = INDOLD + EVOLV
                DII = DF
        IND = IND + EVOLV
        if (IND > NPT):
            print('here')
            break
#         print ("IND, IND2, NPT", IND, IND2, NPT)
        DI = DII
        
    return ZLYAP

In [ ]:
x = f_t(dyn_pendulum, simple_pendulum, 0.001, test.T[209], 500)
le = sample_based_truncated_le (x)

In [ ]:
le

In [ ]:
ra = np.pi
rv = 8
T = 1000
test = np.array([  np.random.uniform(0, 2*np.pi, size=(500)), 
                 np.random.uniform(-rv, rv, size=(500))])
#new_test = np.append(test.T,np.array([[np.pi,0]]),axis=0)
# new_test = new_test.T
new_test_sin = np.array([np.sin(new_test[0]),new_test[1]])
cum_list = []
for i in new_test_sin.T:
    x = f_t(dyn_pendulum, simple_pendulum, 0.001, i, T)
    cum_list.append(sample_based_truncated_le (x))

zs = np.array(cum_list)


In [ ]:
np.argwhere(np.isnan(zs))

In [ ]:
test.T[209]

In [ ]:

fig = plt.figure()
ax = fig.add_subplot(111)

plot1 = ax.scatter(test[0], test[1], 
                   c=zs, cmap=plt.cm.rainbow,vmin=zs.min(), vmax=zs.max())
cbar1 = fig.colorbar(plot1, shrink=0.6, aspect=5)
ax.set_xlabel('angle from the lowest point')
ax.set_ylabel('angular velocity')
plt.title ('le of simple pendulum, at T={}, +- [0,2pi] angle from the lowest point and +- 8 angular velocity'.format(T,ra))
plt.show()

In [ ]:
T = [200, 1000, 1500]
DT = 0.001
X = np.array(np.meshgrid(np.arange(0 , 2*np.pi, 0.2),
                      np.arange(-8 , 8, 0.2)))



In [ ]:
def get_plots (T,X,DT):
    for i in T:
        for j in [i/10, i/2, 0.75*i]:
            evolv_physical = j * DT
            X_reshaped = X.T.reshape(X.T.shape[0]*X.T.shape[1],2)
            cum_list = []
            for k in X_reshaped:
                x = f_t(dyn_pendulum, simple_pendulum, DT, k, i)
                cum_list.append(sample_based_truncated_le (x,evolv_physical,DT,0.15,1e-17))
            zs = np.array(cum_list)
            
            fig = plt.figure(figsize=(15,15))
            ax = fig.add_subplot(111)

            plot1 = ax.scatter(X_reshaped.T[0], X_reshaped.T[1], 
                               c=zs, cmap=plt.cm.plasma,vmin=zs.min(), vmax=zs.max(), s=450)
            cbar1 = fig.colorbar(plot1, shrink=0.6, aspect=5)
            ax.set_xlabel('angle from the lowest point')
            ax.set_ylabel('angular velocity')
            plt.title ('le of simple pendulum, at T={}, +- [0,2pi] angle from the lowest point and +- 8 angular velocity'.format(i))
            plt.savefig(str(i) + "_" + str(j) + ".png")

In [ ]:
get_plots (T,X,DT)

In [ ]:
1500/100

In [ ]:
0.0015/0.001

In [ ]:
T = 1500
X = np.array(np.meshgrid(np.arange(0 , 2*np.pi, 0.2),
                      np.arange(-8 , 8, 0.2)))

X_reshaped = X.T.reshape(X.T.shape[0]*X.T.shape[1],2)
# new_test_sin = np.array([np.sin(X_reshaped.T[0]),X_reshaped.T[1]])
cum_list = []
for i in X_reshaped:
    x = f_t(dyn_pendulum, simple_pendulum, 0.001, i, T)
    cum_list.append(sample_based_truncated_le (x,3,0.001,10,1e-17))
    
zs = np.array(cum_list)

In [ ]:
fig = plt.figure(figsize=(15,15))
ax = fig.add_subplot(111)

plot1 = ax.scatter(X_reshaped.T[0], X_reshaped.T[1], 
                   c=zs, cmap=plt.cm.plasma,vmin=zs.min(), vmax=zs.max(), s=450)
cbar1 = fig.colorbar(plot1, shrink=0.6, aspect=5)
ax.set_xlabel('angle from the lowest point')
ax.set_ylabel('angular velocity')
plt.title ('le of simple pendulum, at T={}, +- [0,2pi] angle from the lowest point and +- 8 angular velocity'.format(T,ra))

In [ ]:
plot1 = ax.pcolormesh(X,Y,zs.reshape(80,32), cmap ='Greens',vmin=zs.min(), vmax=zs.max())
cbar1 = fig.colorbar(plot1, shrink=0.6, aspect=5)
plt.show()

In [ ]:
X_reshaped[np.argwhere(np.isnan(zs))]

In [ ]:
x = f_t(dyn_pendulum, simple_pendulum, 0.001,np.array([np.pi+0.1, 0]), 500)
le = sample_based_truncated_le (x,0.1,0.001,10,1e-16)

In [ ]:
le

In [ ]:
Z = x.T

#find nearest neighbor to the first data point
for i in range(1, int(NPT)):
    D = 0.0
    #iterate through the array, compare first point with the rest
    D = np.linalg.norm(Z[IND] - Z[i])
    if ((D<DI) and (D>=SCALMN)):
        DI = D
        IND2 = i
        
SUM_list = []
while ((IND + EVOLV < NPT) and (IND2 + EVOLV < NPT)): #40
    PT1 = Z[IND+EVOLV] #would this be the dimesion
    PT2 = Z[IND2+EVOLV]
    DF = np.linalg.norm(PT1 - PT2)
    ITS = ITS + 1
    SUM_list.append(SUM)
    SUM = SUM + np.log2(DF/DI)/EVOLV_physical
    ZLYAP = SUM/float(ITS)
    
    INDOLD = IND2
    ZMULT = 1.0
    ANGLMX = 0.3
    #THMIN = 3.14
    while (True): #70
        #print("here")
        THMIN = 3.14

        for i in range (int(NPT)): #100
            III = int(np.absolute(i-(IND+EVOLV)))
            if (III >= 10):
                #find angular change old to new vector
                DNEW = 0.0
                DNEW = np.linalg.norm(PT1 - Z[i])
                if ((DNEW <= ZMULT*SCALMX) and (DNEW >= SCALMN)):
                    DOT = 0.0
                    DOT = np.dot((PT1 - Z[i]),(PT1 - PT2))
                    CTH = np.absolute(DOT/(DNEW*DF))
                    #print(CTH)
                    if (CTH > 1.0):
                        CTH = 1.0
                    TH = np.arccos(CTH)
                    #save smallest angular change so far
                    if TH <= THMIN:
                        THMIN = TH
                        DII = DNEW
                        IND2 = i
        if (THMIN < ANGLMX):
            #print ("here")
            break
        # look at longer distance if cant find replacement
        ZMULT = ZMULT + 1
        if (ZMULT > 5):
            #no replacement at 5*scale, double search angle, reset distance
            ZMULT = 1.0
            ANGLMX = 2.0*ANGLMX
        if (ANGLMX >= 3.14):
            IND2 = INDOLD + EVOLV
            DII = DF
    IND = IND + EVOLV
    if (IND > NPT):
        print('here')
        break
    print ("IND, IND2, NPT", IND, IND2, NPT)
    DI = DII
    
print (SUM)